# Ejercicios de API's
## Enunciado - Obtener datos energéticos usando la API de Red Eléctrica.

Descripción: La empresa para la que trabajamos dispone de una base de datos que contiene información sobre el número de personas censadas en las distintas Comunidades Autónomas españolas desde 1998 hasta 2022. El objetivo es enriquecer estos datos con información sobre la generación de energía renovable y no renovable, utilizando la API de REData, la cual proporciona información sobre el mercado energético en España. Puedes encontrar la documentación de la API aquí: (https://www.ree.es/es/apidatos).

**1. Extracción de Datos a Nivel Nacional:**

Extraer datos de generación de energía renovable y no renovable a nivel nacional desde el 1 de enero de 2011 hasta el 31 de diciembre de 2022. La información debe ser recopilada a nivel mensual.

Transformar los datos para incluir una columna nueva que identifique si la energía es renovable o no renovable.

Almacenar los datos en un DataFrame.

In [30]:
import requests
import pandas as pd

url_ReData = f'https://apidatos.ree.es/es/datos/generacion/estructura-generacion?start_date=2022-01-01T00:00&end_date=2022-12-31T23:59&time_trunc=month'

def obtencion_datos(url_ReData):
    respuesta_api_ReData = requests.get(url_ReData)  
    print (f'La llamada a la API nos ha dado una respuesta de tipo: {respuesta_api_ReData.status_code}')
    
    if respuesta_api_ReData.status_code != 200:
        print(f"La llamada falló con el código de estado {respuesta_api_ReData.reason}")
        return None
    return respuesta_api_ReData.json()


In [31]:
#Llamo a la función y obtengo los datos en formato json de los dos primeros años, divividos en meses.
datos_ReData_periodo_2022 = obtencion_datos (url_ReData)
datos_ReData_periodo_2022

La llamada a la API nos ha dado una respuesta de tipo: 200


{'data': {'type': 'Generación por tecnología',
  'id': 'gen1',
  'attributes': {'title': 'Generación por tecnología',
   'last-update': '2022-12-12T12:48:47.000+01:00',
   'description': None},
  'meta': {'cache-control': {'cache': 'HIT',
    'expireAt': '2024-05-05T08:00:05'}}},
 'included': [{'type': 'Hidráulica',
   'id': '10288',
   'groupId': '1',
   'attributes': {'title': 'Hidráulica',
    'description': None,
    'color': '#0090d1',
    'type': 'Renovable',
    'magnitude': None,
    'composite': False,
    'last-update': '2022-12-12T12:48:38.000+01:00',
    'values': [{'value': 2085857.451,
      'percentage': 0.08582950714262061,
      'datetime': '2022-01-01T00:00:00.000+01:00'},
     {'value': 1174970.17,
      'percentage': 0.05563114753653532,
      'datetime': '2022-02-01T00:00:00.000+01:00'},
     {'value': 1775478.144,
      'percentage': 0.07797540348807623,
      'datetime': '2022-03-01T00:00:00.000+01:00'},
     {'value': 1802751.588,
      'percentage': 0.082799383

In [5]:
#Entro en la lista de diccionarios, para poder crear las keys de mi diccionario con los datos limpios.
datos_ReData_periodo_2022.keys()

dict_keys(['data', 'included'])

In [6]:
#Consigo acceder al tipo de energía y ver que hay 18 tipos diferentes.
len(datos_ReData_periodo_2022["included"])

18

In [7]:
#Consigo acceder al nombre de cada tipo de energía.
datos_ReData_periodo_2022["included"][0]["attributes"]["title"]

'Hidráulica'

In [19]:
#Consigo acceder al NOMBRE de cada energía.
datos_ReData_periodo_2022["included"][0]["attributes"]["title"]

tipo_energia = []

for title in datos_ReData_periodo_2022["included"]:
    tipo_energia.append(title["attributes"]["title"])

tipo_energia

['Hidráulica',
 'Turbinación bombeo',
 'Nuclear',
 'Carbón',
 'Fuel + Gas',
 'Motores diésel',
 'Turbina de gas',
 'Turbina de vapor',
 'Ciclo combinado',
 'Hidroeólica',
 'Eólica',
 'Solar fotovoltaica',
 'Solar térmica',
 'Otras renovables',
 'Cogeneración',
 'Residuos no renovables',
 'Residuos renovables',
 'Generación total']

In [20]:
#Consigo acceder al TIPO de energía y clasificarla en función de si es RENOVABLE o NO ES RENOVABLE.
datos_ReData_periodo_2022["included"][0]["attributes"]["type"]

tipo_generacion = {"Renovable":[], "No Renovable": []}

for type in datos_ReData_periodo_2022["included"]:
    tipo_energia = type["attributes"]["type"]
    if tipo_energia == "Renovable":
        tipo_generacion["Renovable"].append(type)
    elif tipo_energia == "No-Renovable":
        tipo_generacion["No Renovable"].append(type)

tipo_generacion

{'Renovable': [{'type': 'Hidráulica',
   'id': '10288',
   'groupId': '1',
   'attributes': {'title': 'Hidráulica',
    'description': None,
    'color': '#0090d1',
    'type': 'Renovable',
    'magnitude': None,
    'composite': False,
    'last-update': '2022-12-12T12:48:38.000+01:00',
    'values': [{'value': 2085857.451,
      'percentage': 0.08582950714262061,
      'datetime': '2022-01-01T00:00:00.000+01:00'},
     {'value': 1174970.17,
      'percentage': 0.05563114753653532,
      'datetime': '2022-02-01T00:00:00.000+01:00'},
     {'value': 1775478.144,
      'percentage': 0.07797540348807623,
      'datetime': '2022-03-01T00:00:00.000+01:00'},
     {'value': 1802751.588,
      'percentage': 0.08279938338802582,
      'datetime': '2022-04-01T00:00:00.000+02:00'},
     {'value': 1923714.557,
      'percentage': 0.08720971736882878,
      'datetime': '2022-05-01T00:00:00.000+02:00'},
     {'value': 1192915.008,
      'percentage': 0.05129290136302024,
      'datetime': '2022-06-0

In [15]:
#Consigo acceder a los VALORES energéticos. 
datos_ReData_periodo_2022["included"][0]["attributes"]["values"]

valor_energetico = []

for values in datos_ReData_periodo_2022["included"]:
    for item in values["attributes"]["values"]:
        valor_energetico.append(item["value"])

valor_energetico

[2085857.451,
 1174970.17,
 1775478.144,
 1802751.588,
 1923714.557,
 1192915.008,
 1045015.928,
 973565.249,
 1154833.141,
 719585.021,
 1311831.73,
 2750595.903,
 215637.741,
 285079.947,
 273848.153,
 336712.619,
 300017.69,
 271315.011,
 216663.274,
 339437.288,
 304785.767,
 385629.677,
 361663.867,
 485359.059,
 5048424.951,
 4771058.269,
 4766690.052,
 4414715.447,
 4066355.361,
 4459458.308,
 5073367.597,
 5122046.93,
 4847367.123,
 4021338.375,
 4182092.888,
 5161189.944,
 709963.219,
 568629.528,
 705226.125,
 691004.601,
 527154.558,
 833036.37,
 882232.497,
 819592.534,
 632066.719,
 380331.127,
 319685.821,
 693262.892,
 0,
 0,
 210125.165,
 185575.526,
 207451.405,
 205100.239,
 215844.38,
 224846.99,
 242769.399,
 245672.779,
 216066.758,
 202310.788,
 192196.611,
 200241.645,
 34446.764,
 34328.002,
 38861.629,
 34572.36,
 42307.777,
 47734.012,
 79729.885,
 85532.472,
 84422.616,
 79275.334,
 47409.887,
 48856.536,
 117429.802,
 102630.663,
 114410.944,
 103636.366,
 8

In [18]:
#Consigo acceder a los MESES. 
datos_ReData_periodo_2022["included"][0]["attributes"]["values"]

meses = []

for values in datos_ReData_periodo_2022["included"]:
    for item in values["attributes"]["values"]:
        meses.append(item["datetime"])

meses

['2022-01-01T00:00:00.000+01:00',
 '2022-02-01T00:00:00.000+01:00',
 '2022-03-01T00:00:00.000+01:00',
 '2022-04-01T00:00:00.000+02:00',
 '2022-05-01T00:00:00.000+02:00',
 '2022-06-01T00:00:00.000+02:00',
 '2022-07-01T00:00:00.000+02:00',
 '2022-08-01T00:00:00.000+02:00',
 '2022-09-01T00:00:00.000+02:00',
 '2022-10-01T00:00:00.000+02:00',
 '2022-11-01T00:00:00.000+01:00',
 '2022-12-01T00:00:00.000+01:00',
 '2022-01-01T00:00:00.000+01:00',
 '2022-02-01T00:00:00.000+01:00',
 '2022-03-01T00:00:00.000+01:00',
 '2022-04-01T00:00:00.000+02:00',
 '2022-05-01T00:00:00.000+02:00',
 '2022-06-01T00:00:00.000+02:00',
 '2022-07-01T00:00:00.000+02:00',
 '2022-08-01T00:00:00.000+02:00',
 '2022-09-01T00:00:00.000+02:00',
 '2022-10-01T00:00:00.000+02:00',
 '2022-11-01T00:00:00.000+01:00',
 '2022-12-01T00:00:00.000+01:00',
 '2022-01-01T00:00:00.000+01:00',
 '2022-02-01T00:00:00.000+01:00',
 '2022-03-01T00:00:00.000+01:00',
 '2022-04-01T00:00:00.000+02:00',
 '2022-05-01T00:00:00.000+02:00',
 '2022-06-01T0

In [33]:
def limpiar_datos_energia(datos_ReData_periodo_2022, diccionario_energia):
    for values in datos_ReData_periodo_2022["included"]:
        for item in values["attributes"]["values"]:
            tipo_energia = values["type"]
            tipo_generacion = "Renovable" if tipo_energia == "Renovable" else "No renovable"
            fecha = item["datetime"]
            valor_energetico = item["value"]
            
            diccionario_energia["tipo_energia"].append(tipo_energia)
            diccionario_energia["tipo_generacion"].append(tipo_generacion)
            diccionario_energia["valor_energetico"].append(valor_energetico)
            diccionario_energia["meses"].append(fecha)

    return diccionario_energia


In [36]:

def obtencion_datos(url_año):
    respuesta_api_ReData = requests.get(url_año)  
    if respuesta_api_ReData.status_code != 200:
        print(f"La llamada falló con el código de estado {respuesta_api_ReData.reason}")
        return None
    return respuesta_api_ReData.json()

def limpiar_datos_energia(datos_ReData_periodo_año, diccionario_energia):
    for values in datos_ReData_periodo_año["included"]:
        for item in values["attributes"]["values"]:
            tipo_energia = values["type"]
            tipo_generacion = "Renovable" if tipo_energia == "Renovable" else "No renovable"
            fecha = item["datetime"]
            valor_energetico = item["value"]
            
            diccionario_energia["tipo_energia"].append(tipo_energia)
            diccionario_energia["tipo_generacion"].append(tipo_generacion)
            diccionario_energia["valor_energetico"].append(valor_energetico)
            diccionario_energia["meses"].append(fecha)

    return diccionario_energia

url_base = 'https://apidatos.ree.es/es/datos/generacion/estructura-generacion?start_date={year}-01-01T00:00&end_date={year}-12-31T23:59&time_trunc=month'
años = [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

diccionario_energia_total = {"tipo_energia": [], 
                             "tipo_generacion": [],
                             "valor_energetico": [],
                             "meses": []}

for año in años:
    url_año = url_base.format(year=año)
    datos_año = obtencion_datos(url_año)
    if datos_año is not None:
        diccionario_energia_total = limpiar_datos_energia(datos_año, diccionario_energia_total)

df_energia_total = pd.DataFrame(diccionario_energia_total)
df_energia_total    

,tipo_energia,tipo_generacion,valor_energetico,meses
0,Hidráulica,No renovable,5.119512e+06,2011-01-01T00:00:00.000+01:00
1,Hidráulica,No renovable,2.998051e+06,2011-02-01T00:00:00.000+01:00
2,Hidráulica,No renovable,3.910363e+06,2011-03-01T00:00:00.000+01:00
3,Hidráulica,No renovable,3.537938e+06,2011-04-01T00:00:00.000+02:00
4,Hidráulica,No renovable,2.797500e+06,2011-05-01T00:00:00.000+02:00
...,...,...,...,...
2482,Generación total,No renovable,2.528589e+07,2022-08-01T00:00:00.000+02:00
2483,Generación total,No renovable,2.366621e+07,2022-09-01T00:00:00.000+02:00
2484,Generación total,No renovable,2.195664e+07,2022-10-01T00:00:00.000+02:00
2485,Generación total,No renovable,2.152262e+07,2022-11-01T00:00:00.000+01:00


**2. Extracción de Datos por Comunidad Autónoma:**

Extraer datos de generación de energía renovable y no renovable por Comunidad Autónoma.

Utilizar el diccionario cod_comunidades para obtener los códigos de cada comunidad autónoma.

Transformar los datos para incluir una columna que especifique la comunidad autónoma y su identificador.

Almacenar los datos en un DataFrame.

In [73]:
cod_comunidades = {'Ceuta': 8744,
                    'Melilla': 8745,
                    'Andalucía': 4,
                    'Aragón': 5,
                    'Cantabria': 6,
                    'Castilla - La Mancha': 7,
                    'Castilla y León': 8,
                    'Cataluña': 9,
                    'País Vasco': 10,
                    'Principado de Asturias': 11,
                    'Comunidad de Madrid': 13,
                    'Comunidad Foral de Navarra': 14,
                    'Comunitat Valenciana': 15,
                    'Extremadura': 16,
                    'Galicia': 17,
                    'Illes Balears': 8743,
                    'Canarias': 8742,
                    'Región de Murcia': 21,
                    'La Rioja': 20}



In [97]:
url_base_comunidades = 'https://apidatos.ree.es/es/datos/generacion/estructura-generacion?start_date=2022-01-01T00:00&end_date=2022-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids={id_comunidad}'

def obtener_datos_generacion_comunidad(id_comunidad):
    url_comunidad = url_base_comunidades.format(id_comunidad=id_comunidad)
    respuesta = requests.get(url_comunidad)
    if respuesta.status_code == 200:
        return respuesta.json()
    else:
        print(f"Error al obtener los datos para la comunidad con ID {id_comunidad}")
        return None

comunidad =  obtener_datos_generacion_comunidad(20)
comunidad

{'data': {'type': 'Generación por tecnología',
  'id': 'gen1',
  'attributes': {'title': 'Generación por tecnología',
   'last-update': '2022-12-12T12:48:47.000+01:00',
   'description': None},
  'meta': {'cache-control': {'cache': 'HIT',
    'expireAt': '2024-05-05T09:40:53'}}},
 'included': [{'type': 'Hidráulica',
   'id': '10330',
   'groupId': '1',
   'attributes': {'title': 'Hidráulica',
    'description': None,
    'color': '#0090d1',
    'type': 'Renovable',
    'magnitude': None,
    'composite': False,
    'last-update': '2023-01-11T12:06:15.000+01:00',
    'values': [{'value': 11875.04,
      'percentage': 0.06262472991363381,
      'datetime': '2022-01-01T00:00:00.000+01:00'},
     {'value': 7291.198,
      'percentage': 0.04307724160616569,
      'datetime': '2022-02-01T00:00:00.000+01:00'},
     {'value': 12458.382,
      'percentage': 0.05853995161528614,
      'datetime': '2022-03-01T00:00:00.000+01:00'},
     {'value': 14409.094,
      'percentage': 0.08047636244701972,

In [98]:
url_base_comunidades = 'https://apidatos.ree.es/es/datos/generacion/estructura-generacion?start_date=2022-01-01T00:00&end_date=2022-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids={id_comunidad}'

def obtener_datos_generacion_comunidad(id_comunidad):
    url_comunidad = url_base_comunidades.format(id_comunidad=id_comunidad)
    respuesta = requests.get(url_comunidad)
    if respuesta.status_code == 200:
        return respuesta.json()
    else:
        print(f"Error al obtener los datos para la comunidad con ID {id_comunidad}")
        return None
    
datos_completos = []

for comunidad, id_comunidad in cod_comunidades.items():
    datos_comunidad = obtener_datos_generacion_comunidad(id_comunidad)
    if datos_comunidad:
        for data in datos_comunidad['included']:
            data['attributes']['comunidad_autonoma'] = comunidad
            data['attributes']['id_comunidad'] = id_comunidad
            datos_completos.append(data['attributes'])

# Convertir los datos completos en un DataFrame
df_comunidades = pd.DataFrame(datos_completos)
df_comunidades

                                                

,title,description,color,type,magnitude,composite,last-update,values,comunidad_autonoma,id_comunidad
0,Motores diésel,None,#8DA69F,No-Renovable,None,False,2022-02-08T09:13:13.000+01:00,"[{'value': 17218.489, 'percentage': 0.99798831...",Ceuta,8744
1,Turbina de gas,None,#00ccff,No-Renovable,None,False,2023-11-28T14:44:18.000+01:00,"[{'value': 34.708, 'percentage': 0.00201168513...",Ceuta,8744
2,Generación total,None,#2b2e34,Generación total,None,True,2022-02-08T09:13:13.000+01:00,"[{'value': 17253.197, 'percentage': 1, 'dateti...",Ceuta,8744
3,Motores diésel,None,#8DA69F,No-Renovable,None,False,2022-02-08T09:13:13.000+01:00,"[{'value': 16001.846, 'percentage': 0.93878920...",Melilla,8745
4,Turbina de gas,None,#00ccff,No-Renovable,None,False,2023-11-28T14:44:18.000+01:00,"[{'value': 0.504, 'percentage': 2.956844849422...",Melilla,8745
...,...,...,...,...,...,...,...,...,...,...
162,Eólica,None,#6fb114,Renovable,None,False,2022-11-11T00:44:32.000+01:00,"[{'value': 97861.89, 'percentage': 0.516088739...",La Rioja,20
163,Solar fotovoltaica,None,#e48500,Renovable,None,False,2023-11-29T17:53:33.000+01:00,"[{'value': 10301.269, 'percentage': 0.05432522...",La Rioja,20
164,Otras renovables,None,#9a5cbc,Renovable,None,False,2022-05-26T14:38:07.000+02:00,"[{'value': 717.291, 'percentage': 0.0037827371...",La Rioja,20
165,Cogeneración,None,#cfa2ca,No-Renovable,None,False,2022-05-26T14:38:07.000+02:00,"[{'value': 149.672, 'percentage': 0.0007893168...",La Rioja,20
